# Constants

In [3]:
import os
import requests
import zipfile
import subprocess as sbp
from pathlib import Path
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from dataclasses import dataclass
from typing import List, Dict, Optional
import json
# You can place this in a utilities cell/file.
import re
from datetime import datetime, timezone
from typing import Optional, Iterable
import pandas as pd
import json
import yaml
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd # Needed for pd.Timedelta
from IPython.display import clear_output

clear_output(wait=True)
print("Cleared output successfully.")


Cleared output successfully.


In [ ]:
# change it to the path where you wanna put your data
BASE_PATH = Path("")
# BASE_PATH = Path("path_to_your_folder_of_work")

In [3]:
datasets = ["known", "composite", "generated"]

In [4]:
def get_json_files(dataset_name):
    dataset_path = BASE_PATH / dataset_name
    if not dataset_path.exists():
        raise FileNotFoundError(f"Dataset path {dataset_path} does not exist.")
    json_files = list(dataset_path.glob("*.json"))
    return json_files

In [5]:
data_paths = {
    name: get_json_files(name)
    for name in datasets
}

In [6]:
for _, json_files in data_paths.items():
    print(len(json_files))

8
10
20


In [7]:
def load_json_to_df(json_file):
    df = pd.read_json(json_file, lines=True)
    src = df.pop("srcNode").apply(pd.Series).add_prefix("src_")
    dst = df.pop("dstNode").apply(pd.Series).add_prefix("dst_")
    flat_df = pd.concat([df, src, dst], axis=1)
    flat_df["datetime"] = pd.to_datetime(flat_df["timestamp"], unit="s")
    return flat_df

In [10]:
known = {
    f"C{i+1}": load_json_to_df(json_file)
    for i, json_file in tqdm(enumerate(data_paths["known"]), total=len(data_paths["known"]), desc="Loading known dataset")
}


Loading known dataset:  38%|███▊      | 3/8 [3:30:16<5:50:27, 4205.58s/it]








Loading known dataset: 100%|██████████| 8/8 [1:04:24<00:00, 483.10s/it]


In [12]:
known['C1'].shape, known['C2'].shape, known['C3'].shape, known['C4'].shape, known['C5'].shape, known['C6'].shape, known['C7'].shape, known['C8'].shape

((844723, 22),
 (155296, 22),
 (442729, 22),
 (961920, 22),
 (950436, 22),
 (1203013, 22),
 (2072151, 22),
 (607416, 22))

In [15]:
for _, df in known.items():
    print(df['label'].value_counts())

label
benign                                            844674
T1059.001_6efbccc1869e8cd618c0d3ecda407d5f            17
T1547.009_501af516bd8b24fee0c7c650ae5cc861            10
T1566.001_1afaec09315ab71fdfb167175e8a019a             9
T1547.001_1f15ab22c39a9b6bb2bb0d77276dfcb3             5
T1562.001_43e3334362b140924f001b256b229ee5             4
T1564.003_9a2edad4053a2b59fb9167a9bc29e7dc             2
T1204.002_522f3f35cd013e63830fa555495a0081             1
T1055.002_e5bcefee-262d-4568-a261-e8a20855ec81         1
Name: count, dtype: int64
label
benign                                            141172
T1005_720a3356-eee1-4015-9135-0fc08f7eb2d5         14088
T1566.001_1afaec09315ab71fdfb167175e8a019a             9
T1059.001_bfff9006-d1fb-46ce-b173-92cb04e9a031         7
T1021.001_dd67068b052fa553ad4a0ac7d6a5ea89             7
T1547.001_1f15ab22c39a9b6bb2bb0d77276dfcb3             6
T1548.002_665432a4-42e7-4ee1-af19-a9a8c9455d0c         4
T1204.002_522f3f35cd013e63830fa555495a0081        

In [16]:
# check if all dfs in known have timestamp valid
for name, df in known.items():
    if not pd.to_datetime(df['timestamp'], errors='coerce').notna().all():
        print(f"Invalid timestamps found in dataframe {name}")

In [17]:
known['C1'].columns

Index(['relation', 'timestamp', 'label', 'src_UUID', 'src_Name', 'src_Image',
       'src_Cmdline', 'src_Type', 'src_Pid', 'dst_UUID', 'dst_Name',
       'dst_Type', 'dst_Key', 'dst_Value', 'dst_Srcaddress', 'dst_Dstaddress',
       'dst_Port', 'dst_Image', 'dst_Cmdline', 'dst_Pid', 'dst_Path',
       'datetime'],
      dtype='object')

In [18]:
# Build log-style content

def make_log_df(df: pd.DataFrame, include_timestamp_in_content=True) -> pd.DataFrame:
    # Columns to combine (exclude label, and maybe timestamp)
    exclude = {'label'}
    if not include_timestamp_in_content:
        exclude.add('timestamp')
    cols = [c for c in df.columns if c not in exclude]
    
    # Preferred timestamp column (keep as integer; you can convert later)
    out = df[['timestamp','label']].copy()
    
    # For speed: convert selected columns to string only where not null
    def row_fmt(row):
        parts = []
        for c in cols:
            v = row[c]
            if pd.isna(v):
                continue
            # Use ISO for datetime column
            if c == 'datetime':
                v = pd.to_datetime(v).isoformat()
            parts.append(f"{c}={v}")
        return " ".join(parts)
    
    out['content'] = df.apply(row_fmt, axis=1)
    return out

In [21]:
_known = {
    name: None for name in known.keys()
}

In [22]:
_known['C1'] = make_log_df(known['C1'])

In [ ]:
_known['C1'].head()

,timestamp,label,content
0,2022-06-14 02:33:36,benign,relation=QueryAllInformationFile timestamp=202...
1,2022-06-14 02:33:36,benign,relation=CreateFile timestamp=2022-06-14 02:33...
2,2022-06-14 02:33:36,benign,relation=QueryAllInformationFile timestamp=202...
3,2022-06-14 02:33:36,benign,relation=CloseFile timestamp=2022-06-14 02:33:...
4,2022-06-14 02:33:36,benign,relation=CreateFile timestamp=2022-06-14 02:33...


In [ ]:
_known['C1'].to_csv("path_to_your_dataset_folder/prod/C1_log.csv", index=False)

In [25]:
_known['C2'] = make_log_df(known['C2'])

In [26]:
_known['C2'].head()

,timestamp,label,content
0,2022-06-14 23:18:51,benign,relation=RegQueryValue timestamp=2022-06-14 23...
1,2022-06-14 23:18:51,benign,relation=RegQueryValue timestamp=2022-06-14 23...
2,2022-06-14 23:18:51,benign,relation=RegQueryValue timestamp=2022-06-14 23...
3,2022-06-14 23:18:51,benign,relation=RegQueryValue timestamp=2022-06-14 23...
4,2022-06-14 23:18:51,benign,relation=RegQueryValue timestamp=2022-06-14 23...


In [ ]:
_known['C2'].to_csv("path_to_your_dataset_folder/prod/C2_log.csv", index=False)

In [28]:
_known['C3'] = make_log_df(known['C3'])

In [29]:
_known['C3'].head()

,timestamp,label,content
0,2022-06-14 04:20:05,benign,relation=RegCreateKey timestamp=2022-06-14 04:...
1,2022-06-14 04:20:05,benign,relation=RegCloseKey timestamp=2022-06-14 04:2...
2,2022-06-14 04:20:05,benign,relation=RegQueryKey timestamp=2022-06-14 04:2...
3,2022-06-14 04:20:05,benign,relation=RegOpenKey timestamp=2022-06-14 04:20...
4,2022-06-14 04:20:05,benign,relation=RegCreateKey timestamp=2022-06-14 04:...


In [ ]:
_known['C3'].to_csv("path_to_your_dataset_folder/prod/C3_log.csv", index=False)

In [31]:
_known['C4'] = make_log_df(known['C4'])
_known['C5'] = make_log_df(known['C5'])
_known['C6'] = make_log_df(known['C6'])
_known['C7'] = make_log_df(known['C7'])
_known['C8'] = make_log_df(known['C8'])

In [ ]:
_known['C4'].to_csv("path_to_your_dataset_folder/prod/C4_log.csv", index=False)
_known['C5'].to_csv("path_to_your_dataset_folder/prod/C5_log.csv", index=False)
_known['C6'].to_csv("path_to_your_dataset_folder/prod/C6_log.csv", index=False)
_known['C7'].to_csv("path_to_your_dataset_folder/prod/C7_log.csv", index=False)
_known['C8'].to_csv("path_to_your_dataset_folder/prod/C8_log.csv", index=False)

In [ ]:
# let's merge the _known dfs into one and then sorted by timestamp and then save it in prod as known.csv

merged_known = pd.concat([_known['C1'], _known['C2'], _known['C3'], _known['C4'], _known['C5'], _known['C6'], _known['C7'], _known['C8']])
merged_known = merged_known.sort_values(by='timestamp')
merged_known.to_csv("path_to_your_dataset_folder/prod/known.csv", index=False)


In [34]:
merged_known.shape

(7237684, 3)

In [35]:
merged_known['label'].value_counts()

label
benign                                            7208077
T1005_720a3356-eee1-4015-9135-0fc08f7eb2d5          28176
T1046_5a4cb2be-2684-4801-9355-3a90c91e0004           1023
T1219_7dabcbecab0334b115feefab1630f84a                100
T1566.001_1afaec09315ab71fdfb167175e8a019a             72
T1204.002_522f3f35cd013e63830fa555495a0081             52
T1071.001_24c3b7b004401d839a5c337201da3484             42
T1083_52177cc1-b9ab-4411-ac21-2eadc4b5d3b8             21
T1059.001_6efbccc1869e8cd618c0d3ecda407d5f             17
T1547.001_1f15ab22c39a9b6bb2bb0d77276dfcb3             16
T1547.009_501af516bd8b24fee0c7c650ae5cc861             10
T1059.001_bfff9006-d1fb-46ce-b173-92cb04e9a031          7
T1007_9d03c91bdae5a80f17f89c987942b5a8                  7
T1021.001_dd67068b052fa553ad4a0ac7d6a5ea89              7
T1105_0856c235a1d26113d4f2d92e39c9a9f8                  7
T1491_47d08617-5ce1-424a-8cc5-c9c978ce6bf9              6
T1567_3d3d378b0bf66f763b0c9d7a129eb130                  6
T1547.00

In [36]:
# create a new column Label in merged_known, it will be 0 if label is benign and 1 otherwise
merged_known['Label'] = merged_known['label'].apply(lambda x: 0 if x == 'benign' else 1) 

In [ ]:
# save
merged_known.to_csv("path_to_your_dataset_folder/prod/known.csv", index=False)


In [38]:
merged_known['Label'].value_counts()

Label
0    7208077
1      29607
Name: count, dtype: int64

In [39]:
merged_known.columns

Index(['timestamp', 'label', 'content', 'Label'], dtype='object')

In [ ]:
def build_content(df: pd.DataFrame) -> pd.DataFrame:
    def fmt(row):
        # Prefer ISO8601 datetime if present
        if pd.notnull(row.get("datetime")):
            ts = row["datetime"].isoformat()
        else:
            ts = str(row.get("timestamp", ""))
        parts = [
            f"time={ts}",
            f"event={row.get('relation','')}",
            f"srcUUID={row.get('src_UUID','')}",
            f"srcProc={row.get('src_Name','unknown')}({row.get('src_Pid','?')})",
        ]
        if pd.notnull(row.get("src_Image")):
            parts.append(f"srcImage={row['src_Image']}")
        if pd.notnull(row.get("src_Cmdline")):
            parts.append(f"srcCmd=\"{row['src_Cmdline']}\"")
        # Destination specifics
        dst_type = row.get("dst_Type","")
        parts.append(f"dstType={dst_type}")
        if dst_type == "Registry":
            if pd.notnull(row.get("dst_Key")):
                parts.append(f"regKey={row['dst_Key']}")
            if pd.notnull(row.get("dst_Value")):
                parts.append(f"regValue=\"{row['dst_Value']}\"")
        elif dst_type == "Process":
            # If the destination is another process
            if pd.notnull(row.get("dst_Name")):
                parts.append(f"dstProc={row['dst_Name']}({row.get('dst_Pid','?')})")
            if pd.notnull(row.get("dst_Image")):
                parts.append(f"dstImage={row['dst_Image']}")
            if pd.notnull(row.get("dst_Cmdline")):
                parts.append(f"dstCmd=\"{row['dst_Cmdline']}\"")
        elif dst_type in ("File","FileObject"):
            if pd.notnull(row.get("dst_Path")):
                parts.append(f"filePath={row['dst_Path']}")
            elif pd.notnull(row.get("dst_Key")):  # fallback if path stored as Key
                parts.append(f"filePath={row['dst_Key']}")
        elif dst_type == "Network":
            sa = row.get("dst_Srcaddress")
            da = row.get("dst_Dstaddress")
            port = row.get("dst_Port")
            if pd.notnull(sa):
                parts.append(f"srcAddr={sa}")
            if pd.notnull(da):
                parts.append(f"dstAddr={da}")
            if pd.notnull(port):
                parts.append(f"dstPort={port}")
        else:
            # Generic fallback
            for k in ("dst_Key","dst_Name"):
                if pd.notnull(row.get(k)):
                    parts.append(f"{k}={row[k]}")
        return " | ".join(parts)

    df = df.copy()
    df["content"] = df.apply(fmt, axis=1)
    return df

# Apply to your sample DF
_df = build_content(_df)
_df[["content"]].head()